In [1]:
import os
import csv
import cv2
import math
import time
import json
import glob
import shutil
import seaborn as sn
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PBL_detection import *
from tqdm.notebook import tqdm as tqdm

In [2]:
def folder_builder(parent_path,folder_name,Rebuild):
    if os.path.exists(os.path.join(parent_path, folder_name)) and not Rebuild:
        print('"{}" folder is exist,if want to rebuild please modify "Rebuild" parameter.'.format(folder_name))
    else:
        if os.path.exists(os.path.join(parent_path, folder_name)):
            shutil.rmtree(os.path.join(parent_path, folder_name))
        try:
            os.makedirs(os.path.join(parent_path, folder_name))
            print('"{}" folder build successful'.format(folder_name))
        except:
            print('folder build error')

In [3]:
#First time Rebuild = True
Rebuild = False
folder_build_path = './'
sub_folder_name_list = ['preprocess','Data']

if not Rebuild and os.path.exists(folder_build_path):
    print('This folder is exist,if want to rebuild please modify "Rebuild" parameter.')
else:
    for sub_folder_name in sub_folder_name_list:
        folder_builder(folder_build_path,sub_folder_name,Rebuild)
    print('Initial the all folder.')

This folder is exist,if want to rebuild please modify "Rebuild" parameter.


In [2]:
csv_lawrence_path = '../csv_file/HV_stage_majority_20210201.csv'
csv_lawrence_file = pd.read_csv(csv_lawrence_path)
csv_lawrence_file.head()

,folder_name,image_name,tooth_side,stage,bone_loss
0,701509 061919 x,NN_190619_104716_EA3E9,9M,1.0,1.0
1,701509 061919 x,NN_190619_104716_EA3E9,9D,1.0,2.0
2,701509 061919 x,NN_190619_104716_EA3E9,10M,3.0,2.0
3,701509 061919 x,NN_190619_104716_EA3E9,10D,3.0,2.0
4,701509 061919 x,NN_190619_104720_EA3E7,7D,1.0,1.0


In [3]:
all_radiograp_list = list(set(csv_lawrence_file['image_name']))
print('Get ',len(all_radiograp_list),'patients radiograp in lawrence CSV file')
print(all_radiograp_list[0])

Get  2523 patients radiograp in lawrence CSV file
NN_150318_095217_C0ACDC


In [7]:
# source_path_list = glob.glob('/root/notebooks/lawrence_compare_all/ALL_radiograp/*.PNG')

# save_path = './1_o_image'

# count = 0
# count_mis = 0

# for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
#     tareget_img_list = [img_path for img_path in source_path_list if img_name in img_path]
#     if len(tareget_img_list) >= 1:
#         #print(img_name , '> 1')
#         count+=1
#         #shutil.copyfile(tareget_img_list[0], os.path.join(save_path, img_name+'.PNG'))
#     elif len(tareget_img_list) == 0:
#         count_mis+=1
#         #print(img_name , '= 0')
# print(count)
# print(count_mis)


# count = 0
# count_mis = 0
# print('Check')
# #check
# check_path_list = glob.glob('./1_o_image/*.PNG')
# for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
#     tareget_img_list = [img_path for img_path in check_path_list if img_name in img_path]
#     if len(tareget_img_list) < 1:
#         count_mis+=1
#         print(img_name)
# print(count_mis)


2511
12
Check


NN_180426_104742_16EDB4
NN_141121_093244_16ED98
NN_141121_093345_16ED97
 NN_180921_142943_9B1076
NN_180426_100748_16EDAB
NN_180426_100749_16EDB9
NN_171019_153422_C07107
NN_180426_104705_16EDAD
NN_180426_104708_16EDB7
NN_171019_155156_C07101
NN_141121_093257_16ED95
NN_180426_100727_16EDBE

12


In [6]:
# source_path_list = glob.glob('/root/notebooks/lawrence_compare_all/PBL_detection/1_seg_image/*.PNG')

# save_path = './1_seg_image'

# count = 0
# count_mis = 0

# for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
#     tareget_img_list = [img_path for img_path in source_path_list if img_name in img_path]
#     if len(tareget_img_list) >= 1:
#         #print(img_name , '> 1')
#         count+=1
#         #shutil.copyfile(tareget_img_list[0], os.path.join(save_path, img_name+'.PNG'))
#     elif len(tareget_img_list) == 0:
#         count_mis+=1
#         #print(img_name , '= 0')
# print(count)
# print(count_mis)


# count = 0
# count_mis = 0
# seg_loss_list= []
# print('Check')
# #check
# check_path_list = glob.glob('./1_seg_image/*.PNG')
# for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
#     tareget_img_list = [img_path for img_path in check_path_list if img_name in img_path]
#     if len(tareget_img_list) < 1:
#         count_mis+=1
#         seg_loss_list.append(img_name)
#         print(img_name)
# print(count_mis)


2511
12
Check


NN_180426_104742_16EDB4
NN_141121_093244_16ED98
NN_141121_093345_16ED97
 NN_180921_142943_9B1076
NN_180426_100748_16EDAB
NN_180426_100749_16EDB9
NN_171019_153422_C07107
NN_180426_104705_16EDAD
NN_180426_104708_16EDB7
NN_171019_155156_C07101
NN_141121_093257_16ED95
NN_180426_100727_16EDBE

12


In [9]:
source_path_list = glob.glob('/root/notebooks/lawrence_compare_all/mask_0505_size/*.PNG')
print(len(source_path_list))
save_path = './1_roots_mask'

count = 0
count_r_name = 0
count_root = 0
count_miss_root = 0

for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
    tareget_img_list = [img_path_ for img_path_ in source_path_list if img_name in img_path_]
    if len(tareget_img_list) >= 1:
        #print(img_name , '> 1')
        count+=1
        for root_img in tareget_img_list:
            count_root+=1
            shutil.copyfile(root_img, os.path.join(save_path, root_img.split('/')[-1]))
    elif len(tareget_img_list) == 0:
        count_miss_root+=1
        #print(img_name , '= 0')
print(count)
print(count_root)
print(count_miss_root)

count = 0
count_mis = 0
seg_loss_list= []
print('Check')
#check
check_path_list = glob.glob('./1_roots_mask/*.PNG')
for img_name in tqdm(all_radiograp_list, total = len(all_radiograp_list)):
    tareget_img_list = [img_path for img_path in check_path_list if img_name in img_path]
    if len(tareget_img_list) < 1:
        count_mis+=1
        seg_loss_list.append(img_name)
        #print(img_name)
print(count_mis)

9051



2498
9051
25
Check



25


In [44]:
print(len(seg_loss_list))

106


In [77]:
# source_path_list = glob.glob('/root/notebooks/lawrence_compare_all/PBL_detection/1_o_image/*.PNG')
# print(len(source_path_list))
# save_path = '/root/notebooks/lawrence_compare_all/o_img_1024'

# count+=0

# for img_name in tqdm(seg_loss_list, total=len(seg_loss_list)):
#     target_list = [img_path for img_path in source_path_list if img_name in img_path]
#     if len(target_list)>0:
#         count+=1
#         o_img = cv2.imread(target_list[0])
#         o_img_resize = cv2.resize(o_img,(1024,1024))
#         cv2.imwrite(os.path.join(save_path,target_list[0].split('/')[-1]),o_img_resize)
    
# print(count)

In [55]:
# source_path_list = glob.glob('/root/notebooks/tooth_thsis_experiment/PBL_detection/preprocess/seg_img_1024/*.PNG')
# print(len(source_path_list))
# save_path = '../seg'

# count=0
# count_miss=0 

# for img_name in tqdm(seg_loss_list, total=len(seg_loss_list)):
#     target_list = [img_path for img_path in source_path_list if img_name in img_path]
#     if len(target_list)>0:
#         count+=1
#         shutil.copyfile(target_list[0], os.path.join(save_path,target_list[0].split('/')[-1]))
#     else:
#         count_miss+1
# print(count)
# print(count_miss)

4963



94
0


In [5]:
# patient_list = glob.glob('./Data/patient_folder/*')
# file_list = glob.glob('./Data/patient_folder/*/*/*.PNG')
    

# show_patient_info(patient_list,file_list)
# assert len(file_list)>0 ,"Loading data error"

In [6]:
# #First time Rebuild = True
# Rebuild = False
# folder_build_path = './preprocess'
# sub_folder_name_list = ['o_img','seg_img','root_mask_img']

# if not Rebuild and os.path.exists(folder_build_path):
#     print('This folder is exist,if want to rebuild please modify "Rebuild" parameter.')
# else:
#     for sub_folder_name in sub_folder_name_list:
#         folder_builder(folder_build_path,sub_folder_name,Rebuild)
#     print('Initial the all folder.')

In [7]:
# o_img_save_path = './preprocess/o_img'

# for img_path in tqdm(file_list,total=len(file_list)):
#     img_name = img_path.split('/')[-1]
#     shutil.copyfile(img_path, os.path.join(o_img_save_path,img_name))

# copy predict segmentation image & resize

In [71]:
seg_img_path = '/root/notebooks/lawrence_compare_all/seg_1024/*.PNG'
seg_img_path_list = glob.glob(seg_img_path)
seg_img_path_list.sort()

o_img_path = '/root/notebooks/lawrence_compare_all/PBL_detection/1_o_image/*.PNG'
o_img_path_list = glob.glob(o_img_path)
# o_img_path_list_FM =[path for path in o_img_path_list if 'FM' not in path]
# o_img_path_list_FM.sort()

print(len(seg_img_path_list),len(o_img_path_list))

94 2511


In [79]:
# save_path = './1_seg_image'
# count = 0
# for seg_img_path in tqdm(seg_img_path_list,total=len(seg_img_path_list)):
#     img_name = seg_img_path.split('/')[-1][:-4]
#     if os.path.isfile(os.path.join(save_path,img_name)):
#         continue
#     o_img_path = [img_path for img_path in o_img_path_list if img_name in img_path]
#     if len(o_img_path)>0:
#         count+=1
#         o_img = cv2.imread(o_img_path[0])
#         seg_img = cv2.imread(seg_img_path)
#         seg_img_resize = cv2.resize(seg_img,(o_img.shape[1],o_img.shape[0]),interpolation = cv2.INTER_NEAREST)
#         cv2.imwrite(os.path.join(save_path, img_name+'.PNG'),seg_img_resize)
#     else:
#         print(img_name)
# print(count)


1


In [10]:
# save_path = './1_seg_image'

# for seg_img_path in tqdm(seg_img_path_list,total=len(seg_img_path_list)):
#     img_name = seg_img_path.split('/')[-1]
#     o_img_path = [img_path for img_path in o_img_path_list if img_name in img_path]
#     o_img = cv2.imread(o_img_path[0])
#     seg_img = cv2.imread(seg_img_path)
#     seg_img_resize = cv2.resize(seg_img,(o_img.shape[1],o_img.shape[0]),interpolation = cv2.INTER_NEAREST)
#     cv2.imwrite(os.path.join(save_path,img_name),seg_img_resize)

In [11]:
# seg_img_path = './preprocess/seg_img/*.PNG'
# seg_img_path_list = glob.glob(seg_img_path)
# seg_img_path_list.sort()

# o_img_path = './Data/patient_folder/*/*/*.PNG'
# o_img_path_list = glob.glob(o_img_path)
# o_img_path_list_FM =[path for path in o_img_path_list if 'FM' not in path]
# o_img_path_list_FM.sort()

# print(len(seg_img_path_list),len(o_img_path_list_FM))

In [12]:
# save_path = './preprocess/seg_img_resize'

# for seg_img_path in tqdm(seg_img_path_list,total=len(seg_img_path_list)):
#     img_name = seg_img_path.split('/')[-1]
#     o_img_path = [img_path for img_path in o_img_path_list if img_name in img_path]
#     o_img = cv2.imread(o_img_path[0])
#     seg_img = cv2.imread(seg_img_path)
#     seg_img_resize = cv2.resize(seg_img,(o_img.shape[1],o_img.shape[0]),interpolation = cv2.INTER_NEAREST)
#     cv2.imwrite(os.path.join(save_path,img_name),seg_img_resize)

In [13]:
# save_path = './preprocess/seg_img_resize_quator'

# for seg_img_path in tqdm(seg_img_path_list,total=len(seg_img_path_list)):
#     img_name = seg_img_path.split('/')[-1]
#     o_img_path = [img_path for img_path in o_img_path_list if img_name in img_path]
#     o_img = cv2.imread(o_img_path[0])
#     seg_img = cv2.imread(seg_img_path)
#     seg_img_resize = cv2.resize(seg_img,(int(o_img.shape[1]/4),int(o_img.shape[0]/4)),interpolation = cv2.INTER_NEAREST)
#     cv2.imwrite(os.path.join(save_path,img_name),seg_img_resize)

In [14]:
# root_img_path = './preprocess/mask/*.png'
# root_img_path_list = glob.glob(root_img_path)
# root_img_path_list.sort()
# print(len(root_img_path_list))
# root_img_path_list_name = [img_path.split('/')[-1][:-6] for img_path in root_img_path_list] 
# print(root_img_path_list_name[0])
# root_img_path_list_name_set = list(set(root_img_path_list_name))
# print(len(root_img_path_list_name_set))

In [15]:
# save_path = './preprocess/root_mask_img'

# for root_img_path in tqdm(root_img_path_list,total=len(root_img_path_list)):
#     img_name = root_img_path.split('/')[-1]
#     #print(img_name)
#     o_img_path = [img_path for img_path in o_img_path_list if img_name[:-6] in img_path]
#     if o_img_path != []:
#         o_img = cv2.imread(o_img_path[0])
#         root_img = cv2.imread(root_img_path)
#         root_img_resize = cv2.resize(root_img,(o_img.shape[1],o_img.shape[0]),interpolation = cv2.INTER_NEAREST)
#         cv2.imwrite(os.path.join(save_path,img_name[:-3]+'PNG'),root_img_resize)